In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Read the CSV file
df = pd.read_csv('R3.csv')
df.head(110)

,Test Cases Name,Test Cases,Requirements,Core Business Functions,Allied Atomic Functions,Technical Functions,Enabling Operations,Encapsulated Functions,Release3/ Fault Count,Fault Severity,Dependency/cross cutting,Verdict
0,test case addition failed in tester's log test...,T1,R10,10,0,0,0,0,0,0,3,1
1,add bugs in tester's log test suite,T2,R10,10,0,0,0,0,0,0,3,1
2,add personal projects in tester's portfolio,T3,R9,10,0,0,0,0,0,0,3,1
3,add requirement button on client project initi...,T4,R6,10,0,0,0,0,0,0,3,1
4,add skills in tester's portfolio,T5,R9,10,0,0,0,0,0,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99,upload resume button in tester's portfolio,T100,R9,10,0,0,0,0,0,0,3,1
100,write delivery and description in tester's por...,T101,R9,10,0,0,0,0,0,0,3,1
101,make payment (admin to tester)dynamic payment ...,T102,R15,0,0,0,0,0,0,0,3,1
102,admin to tester secure payment,T103,R17,0,0,0,0,0,0,0,3,1


In [3]:
# Get the shape of the DataFrame
shape = df.shape
# Print the number of rows and columns
print("Number of rows:", shape[0])
print("Number of columns:", shape[1])

df.columns

Number of rows: 104
Number of columns: 12


Index(['Test Cases Name', 'Test Cases', 'Requirements',
       'Core Business Functions', 'Allied Functions', 'Technical Functions',
       'Atomic Operations', 'Encapsulated Functions', 'Release3/ Fault Count',
       'Fault Severity', 'Dependency/cross cutting', 'Verdict'],
      dtype='object')

In [4]:
# Extract the test cases from the CSV
test_cases = df['Test Cases'] +', ' + df['Requirements']

# Extract the feature columns from the CSV
feature_columns = df.columns[3:]

# Create a dictionary to store the features and their values
features = {}
for feature in feature_columns:
    features[feature] = df[feature].values

In [5]:
# Define the environment
class TestPrioritizationEnv:
    def __init__(self, test_cases, features):
        self.test_cases = test_cases
        self.features = features
        self.state_size = len(features)  # Change state_size to match the number of features
        self.action_size = len(test_cases)
        self.current_test = 0

    def reset(self):
        self.current_test = 0
        return self.get_state()
    
    def get_state(self):
        if self.current_test < len(self.test_cases):
            state = [self.features[feature][self.current_test] for feature in self.features]
            return state
        else:
           
            return None

    def step(self, action):
        test_case = self.test_cases[self.current_test]
        reward = self.calculate_business_value(test_case)
        self.current_test += 1
        done = self.current_test == len(self.test_cases)
        return self.get_state(), reward, done

    def calculate_business_value(self, test_case):
        index = self.test_cases.tolist().index(test_case)
        business_value = sum([self.features[feature][index] for feature in self.features])
        return business_value

In [6]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.learning_rate = learning_rate
        self.model = self.build_model()
    
    def build_model(self):
        model = Sequential()
        model.add(Dense(self.state_size, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

  
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))  

    
    def act(self, state):
        state = np.array(state).reshape(1, -1)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
 
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = np.array(state)  
            state = np.reshape(state, [1, self.state_size])

            if next_state is not None:
                next_state = np.array(next_state)  
                next_state = np.reshape(next_state, [1, self.state_size])
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            else:
               
                target = reward  

            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)


    def train(self, env, episodes, batch_size):
        for episode in range(episodes):
            state = env.reset()
            total_reward = 0
            done = False
            while not done:
                action = self.act(state)
                next_state, reward, done = env.step(action)
                self.remember(state, action, reward, next_state, done)
                total_reward += reward
                state = next_state

            if len(self.memory) > batch_size:
                self.replay(batch_size)

    def get_prioritized_test_cases(self, env):
        prioritized_test_cases = []
        state = env.reset()
        done = False
        while not done:
            action = self.act(state)
            test_case = env.test_cases[env.current_test]
            reward = env.calculate_business_value(test_case)
            prioritized_test_cases.append((test_case, reward))
            next_state, _, done = env.step(action)
            state = next_state
        return prioritized_test_cases

In [7]:
# Create the environment with the test cases and features
env = TestPrioritizationEnv(test_cases, features)

# Create the DQN agent
state_size = env.state_size
action_size = env.action_size
agent = DQNAgent(state_size, action_size)

# Train the DQN agent
episodes =10 
batch_size = 32
agent.train(env, episodes, batch_size)

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 31ms/step


In [8]:
# Calculate the business value and prioritize the test cases
prioritized_test_cases = []
for test_case in test_cases:
    business_value = env.calculate_business_value(test_case)
    prioritized_test_cases.append((test_case, business_value))

# Sort the test cases based on business value (in descending order)
prioritized_test_cases.sort(key=lambda x: x[1], reverse=True)

# Print the prioritized test cases
for test_case, business_value in prioritized_test_cases:
    print(f"Test Case: {test_case}, Business Value: {business_value}")

Test Case: T1, R10, Business Value: 14
Test Case: T2, R10, Business Value: 14
Test Case: T3, R9, Business Value: 14
Test Case: T4, R6, Business Value: 14
Test Case: T5, R9, Business Value: 14
Test Case: T6, R9, Business Value: 14
Test Case: T7, R7, Business Value: 14
Test Case: T8, R7, Business Value: 14
Test Case: T12, R7, Business Value: 14
Test Case: T13, R7, Business Value: 14
Test Case: T27, R6, Business Value: 14
Test Case: T41, R6, Business Value: 14
Test Case: T50, R6, Business Value: 14
Test Case: T78, R10, Business Value: 14
Test Case: T81, R10, Business Value: 14
Test Case: T84, R10, Business Value: 14
Test Case: T88, R4, Business Value: 14
Test Case: T100, R9, Business Value: 14
Test Case: T101, R9, Business Value: 14
Test Case: T18, R3, Business Value: 12
Test Case: T38, R1, Business Value: 12
Test Case: T44, R1, Business Value: 12
Test Case: T45, R3, Business Value: 12
Test Case: T55, R16, Business Value: 12
Test Case: T56, R16, Business Value: 12
Test Case: T57, R16, Bus

In [9]:
# Calculate the business value and prioritize the test cases
prioritized_test_cases = []
prioritized_requirements = set()
for test_case in test_cases:
    business_value = env.calculate_business_value(test_case)
    requirement = test_case.split(',')[1].strip()
    if requirement not in prioritized_requirements:
        prioritized_test_cases.append((test_case, business_value))
        prioritized_requirements.add(requirement)

# Sort the test cases based on business value (in descending order)
prioritized_test_cases.sort(key=lambda x: x[1], reverse=True)

# Print the prioritized test cases
for test_case, business_value in prioritized_test_cases:
    print(f"Test Case: {test_case}, Business Value: {business_value}")

Test Case: T1, R10, Business Value: 14
Test Case: T3, R9, Business Value: 14
Test Case: T4, R6, Business Value: 14
Test Case: T7, R7, Business Value: 14
Test Case: T88, R4, Business Value: 14
Test Case: T18, R3, Business Value: 12
Test Case: T38, R1, Business Value: 12
Test Case: T15, R5, Business Value: 11
Test Case: T14, R14, Business Value: 10
Test Case: T17, R2, Business Value: 10
Test Case: T37, R8, Business Value: 10
Test Case: T54, R16, Business Value: 9
Test Case: T11, R11, Business Value: 8
Test Case: T9, R12, Business Value: 6
Test Case: T16, R13, Business Value: 5
Test Case: T102, R15, Business Value: 4
Test Case: T103, R17, Business Value: 4


In [10]:
# Scale the feature values using MinMaxScaler
scaler = MinMaxScaler()
for feature in feature_columns:
    feature_values = features[feature].reshape(-1, 1)
    scaled_values = scaler.fit_transform(feature_values).flatten()
    features[feature] = scaled_values
# Calculate the business value and prioritize the test cases
prioritized_test_cases = []
for test_case in test_cases:
    business_value = env.calculate_business_value(test_case)
    prioritized_test_cases.append((test_case, business_value))

# Sort the test cases based on business value (in descending order)
prioritized_test_cases.sort(key=lambda x: x[1], reverse=True)

# Print the prioritized test cases
for test_case, business_value in prioritized_test_cases:
    print(f"Test Case: {test_case}, Business Value: {business_value}")

Test Case: T33, R11, Business Value: 4.0
Test Case: T15, R5, Business Value: 3.5
Test Case: T97, R14, Business Value: 3.5
Test Case: T1, R10, Business Value: 3.0
Test Case: T2, R10, Business Value: 3.0
Test Case: T3, R9, Business Value: 3.0
Test Case: T4, R6, Business Value: 3.0
Test Case: T5, R9, Business Value: 3.0
Test Case: T6, R9, Business Value: 3.0
Test Case: T7, R7, Business Value: 3.0
Test Case: T8, R7, Business Value: 3.0
Test Case: T9, R12, Business Value: 3.0
Test Case: T10, R12, Business Value: 3.0
Test Case: T11, R11, Business Value: 3.0
Test Case: T12, R7, Business Value: 3.0
Test Case: T13, R7, Business Value: 3.0
Test Case: T14, R14, Business Value: 3.0
Test Case: T18, R3, Business Value: 3.0
Test Case: T19, R13, Business Value: 3.0
Test Case: T27, R6, Business Value: 3.0
Test Case: T28, R5, Business Value: 3.0
Test Case: T29, R5, Business Value: 3.0
Test Case: T30, R5, Business Value: 3.0
Test Case: T31, R11, Business Value: 3.0
Test Case: T32, R11, Business Value: 3.

In [11]:
# Calculate the business value and prioritize the test cases
prioritized_test_cases = []
prioritized_requirements = set()
for test_case in test_cases:
    business_value = env.calculate_business_value(test_case)
    requirement = test_case.split(',')[1].strip()
    if requirement not in prioritized_requirements:
        prioritized_test_cases.append((test_case, business_value))
        prioritized_requirements.add(requirement)

# Sort the test cases based on business value (in descending order)
prioritized_test_cases.sort(key=lambda x: x[1], reverse=True)

# Print the prioritized test cases
for test_case, business_value in prioritized_test_cases:
    print(f"Test Case: {test_case}, Business Value: {business_value}")

Test Case: T15, R5, Business Value: 3.5
Test Case: T1, R10, Business Value: 3.0
Test Case: T3, R9, Business Value: 3.0
Test Case: T4, R6, Business Value: 3.0
Test Case: T7, R7, Business Value: 3.0
Test Case: T9, R12, Business Value: 3.0
Test Case: T11, R11, Business Value: 3.0
Test Case: T14, R14, Business Value: 3.0
Test Case: T18, R3, Business Value: 3.0
Test Case: T37, R8, Business Value: 3.0
Test Case: T88, R4, Business Value: 3.0
Test Case: T16, R13, Business Value: 2.5
Test Case: T38, R1, Business Value: 2.4
Test Case: T17, R2, Business Value: 2.375
Test Case: T102, R15, Business Value: 2.0
Test Case: T103, R17, Business Value: 2.0
Test Case: T54, R16, Business Value: 1.875


In [12]:
# Get the unique requirements and test cases
requirements = df['Requirements'].unique()
test_cases = df['Test Cases'].unique()

# Create an empty traceability matrix
traceability_matrix = pd.DataFrame(columns=['Requirements'] + list(test_cases))

# Populate the traceability matrix
rows = []
for requirement in requirements:
    req_data = df[df['Requirements'] == requirement]
    row = [requirement] + [test_case in req_data['Test Cases'].values for test_case in test_cases]
    rows.append(row)

# Convert the list of rows to a DataFrame
traceability_matrix = pd.DataFrame(rows, columns=traceability_matrix.columns)

# Print the traceability matrix
print(traceability_matrix)

   Requirements     T1     T2     T3     T4     T5     T6     T7     T8  \
0           R10   True   True  False  False  False  False  False  False   
1            R9  False  False   True  False   True   True  False  False   
2            R6  False  False  False   True  False  False  False  False   
3            R7  False  False  False  False  False  False   True   True   
4           R12  False  False  False  False  False  False  False  False   
5           R11  False  False  False  False  False  False  False  False   
6           R14  False  False  False  False  False  False  False  False   
7            R5  False  False  False  False  False  False  False  False   
8           R13  False  False  False  False  False  False  False  False   
9            R2  False  False  False  False  False  False  False  False   
10           R3  False  False  False  False  False  False  False  False   
11           R8  False  False  False  False  False  False  False  False   
12           R1  False  F

In [13]:
# Get the unique requirements and test cases
requirements = df['Requirements'].unique()
test_cases = df['Test Cases'].unique()

# Create an empty traceability matrix
traceability_matrix = pd.DataFrame(columns=['Requirements'] + list(test_cases))

# Populate the traceability matrix
rows = []
for requirement in requirements:
    req_data = df[df['Requirements'] == requirement]
    row = [requirement] + [int(test_case in req_data['Test Cases'].values) for test_case in test_cases]
    rows.append(row)

# Convert the list of rows to a DataFrame
traceability_matrix = pd.DataFrame(rows, columns=traceability_matrix.columns)

# Print the traceability matrix
print(traceability_matrix)

   Requirements  T1  T2  T3  T4  T5  T6  T7  T8  T9  ...  T95  T96  T97  T98  \
0           R10   1   1   0   0   0   0   0   0   0  ...    0    0    0    0   
1            R9   0   0   1   0   1   1   0   0   0  ...    0    0    0    0   
2            R6   0   0   0   1   0   0   0   0   0  ...    0    0    0    0   
3            R7   0   0   0   0   0   0   1   1   0  ...    0    0    0    0   
4           R12   0   0   0   0   0   0   0   0   1  ...    0    0    0    0   
5           R11   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
6           R14   0   0   0   0   0   0   0   0   0  ...    0    0    1    0   
7            R5   0   0   0   0   0   0   0   0   0  ...    0    0    0    0   
8           R13   0   0   0   0   0   0   0   0   0  ...    0    0    0    1   
9            R2   0   0   0   0   0   0   0   0   0  ...    0    1    0    0   
10           R3   0   0   0   0   0   0   0   0   0  ...    1    0    0    0   
11           R8   0   0   0   0   0   0 